# (Py)SparkSQL Exercizes

----
###Try to solve the following exercize

In [ ]:
FILE_PATH = "file:///notebooks/cineca/data/"

In [ ]:
'''
A SQLConext wraps the SparkContext,
and adds functions for working with structured data.
'''
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlCtx = SQLContext(sc)
#sqlCtx.sql("SET spark.sql.parquet.binaryAsString=true")
#sqlCtx.sql("SET spark.sql.inMemoryColumnarStorage.compressed=true")
sqlCtx.sql("SET spark.sql.parquet.useDataSourceApi=false")

Now we can load a set of data in that is stored in the Parquet format.
Parquet is a self-describing columnar format. Since it is self-describing,
Spark SQL will automatically be able to infer all of the column names and their datatypes. 

In [ ]:
%%bash
hdfs dfs -mkdir /wiki

In [ ]:
%%bash
hdfs dfs -ls /

In [ ]:
%%bash
hdfs dfs -put localfile /notebooks/cineca/data/wiki_parquet /wiki

In [ ]:
%%bash
hdfs dfs -ls /wiki

In [ ]:
%%bash
#hdfs dfs -rmr /wiki

In [ ]:
wikiData = sqlCtx.parquetFile("/wiki/wiki_parquet")

The result of loading in a parquet file is a SchemaRDD.
A SchemaRDD has all of the functions of a normal RDD.
For example, lets figure out how many records are in the data set.

In [ ]:
wikiData.count()

In addition to standard RDD operatrions, SchemaRDDs also have
extra information about the names and types of the columns in the dataset.
This extra schema information makes it possible to run SQL queries against
the data after you have registered it as a table.

In [ ]:
# to describe the schema
wikiData.printSchema()

In [ ]:
%%time
# number of record (again)
result_df = sqlCtx.sql("SELECT COUNT(*) AS pageCount FROM wikiData")
result_df

In [ ]:
result = result_df.collect()
result[0].pageCount

In [ ]:
result_df.toPandas()

SQL can be a powerfull tool from performing complex aggregations.
For example, the following query returns the top 10 usersnames by the number of pages
they created.

```
sqlCtx.sql("SELECT username, COUNT(*) AS cnt FROM wikiData WHERE username <> '' GROUP BY username ORDER BY cnt DESC LIMIT 10").collect()

```

In [ ]:
sqlCtx.sql("SELECT username, COUNT(*) AS cnt\
            FROM wikiData\
            WHERE username <> ''\
            GROUP BY username\
            ORDER BY cnt DESC LIMIT 10")\
    .toPandas()

Using the `LIKE` operator (inside the SQL query),
select the id and title of the wiki articles
that contain both **italy** and **bologna** in its text.

In [ ]:
wikiData.printSchema()

In [ ]:
%%time
#sqlCtx.sql("SELECT id AS ID, title AS Title FROM wikiData WHERE test LIKE 'bologna' LIMIT 10").toPandas()
sqlCtx.sql("SELECT id, title\
            FROM wikiData\
            WHERE text LIKE '%bologna%'\
            AND text LIKE '%italy%'\
            LIMIT 10")\
    .toPandas()

In [ ]:
#%reset